In [1]:
import sentence_transformers
import pandas as pd

/Volumes/External/opt/anaconda3/envs/ollama2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from langchain.vectorstores import Chroma
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.chains import VectorDBQA
import sentence_transformers
import pandas as pd

In [3]:
%cd /Volumes/External/source/GTPracticum/data 
%ls -salh

/Volumes/External/source/GTPracticum/data
total 13160432
      0 drwxr-xr-x   25 kariato  staff   800B Mar 16 22:20 ./
      0 drwxr-xr-x    8 kariato  staff   256B Mar 15 14:14 ../
     16 -rw-r--r--@   1 kariato  staff   6.0K Feb 10 14:58 .DS_Store
  27280 -rw-r--r--@   1 kariato  staff    13M Feb 21 00:04 2023-10-eb-big-book-of-mlops-2nd-edition-v2-102723-final (1).pdf
  27280 -rw-r--r--@   1 kariato  staff    13M Feb 20 10:15 2023-10-eb-big-book-of-mlops-2nd-edition-v2-102723-final.pdf
    464 -rw-r--r--@   1 kariato  staff   232K Feb 10 16:00 9deec473-6ab3-4c90-a2ec-86ae93b0a6e6.pdf
   1472 -rw-r--r--@   1 kariato  staff   730K Mar 16 22:20 W-2_Form_2023_Davey_2024_01_05_13_13_13_-0800_W-2_ESS (1).pdf
   1472 -rw-r--r--@   1 kariato  staff   730K Mar 16 22:15 W-2_Form_2023_Davey_2024_01_05_13_13_13_-0800_W-2_ESS.pdf
 733000 -rw-r--r--@   1 kariato  staff   358M Feb 10 14:56 archive (1).zip
      0 drwxr-xr-x    4 kariato  staff   128B Mar  5 11:36 chroma2_db/
      0 drwxr-xr-x   

/Volumes/External/opt/anaconda3/envs/ollama2/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [4]:
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
embedding_function2 = SentenceTransformerEmbeddings(model_name="all-mpnet-base-v2")

/Volumes/External/opt/anaconda3/envs/ollama2/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [5]:
db = Chroma(embedding_function=embedding_function,persist_directory="./chroma_db")
db2 = Chroma(embedding_function=embedding_function2,persist_directory="./chroma2_db")
db3 = Chroma(embedding_function=embedding_function2,persist_directory="./chroma3_db")
#qa = VectorDBQA.from_chain_type(llm=OpenAI(), chain_type="stuff", vectorstore=vectordb)

In [6]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

In [7]:
import re
def cleanText(text):
        
    text = text.replace('\\n','')
    text = text.replace('\\','')
    text = re.sub(' +', ' ', text)
    text = text.replace('\t', '')
    text = text.replace('\n', '')
    return text

In [8]:
query = "mark to market accounting offsetting concerns hiding losses"
docs = db2.similarity_search_with_score(query,k=4000)
topics=[]
for doc in docs:
    details = cleanText(doc[0].page_content)
    if details not in topics:
        topics.append(details)
#topics

In [19]:
doc[0].metadata['SENT']

' Monday, April 30, 2001 10>>  Wednesday, April 25, 2001 7'

In [17]:
type(topics[1])

str

In [18]:
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords

stopwords = list(stopwords.words('english')) + ['http', 'https', 'amp', 'com']

embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

In [19]:
from bertopic import BERTopic
model = BERTopic(embedding_model=embedding_model)
topics2, probs = model.fit_transform(topics)

In [20]:
model.visualize_barchart()

In [21]:
model.visualize_hierarchy()

In [22]:
model.visualize_heatmap()

In [23]:
freq = model.get_topic_info()
print("Number of topics: {}".format( len(freq)))
freq

Number of topics: 27


,Topic,Count,Name,Representation,Representative_Docs
0,-1,391,-1_the_to_of_and,"[the, to, of, and, is, we, in, on, this, that]",[Subject: FW: Bridgeline Storage & TransportRi...
1,0,126,0_the_of_to_in,"[the, of, to, in, accounting, that, and, finan...",[In an age when giant earnings write-offs have...
2,1,39,1_the_canada_to_is,"[the, canada, to, is, curve, for, in, you, we,...","[Subject: Re: Canada p&lRobin,$4.1 M was to ha..."
3,2,34,2_ect_to_the_of,"[ect, to, the, of, and, enron, with, we, are, in]",[Subject: RE: Sleeves/bookoutsWe should condit...
4,3,28,3_fuel_tw_the_to,"[fuel, tw, the, to, in, we, that, for, would, ...",[In the TW proposal the cash inflows are the r...
5,4,27,4_budget_the_to_cost,"[budget, the, to, cost, and, costs, 2001, we, ...",[Subject: Re: Revenue Management Project / 200...
6,5,26,5_3d_million_net_of,"[3d, million, net, of, income, and, br, tax, s...","[121,012 102,382 General and administrative 19..."
7,6,22,6_deal_month_pma_you,"[deal, month, pma, you, for, this, the, to, mi...",[Subject: RE: PMA's - Tacoma DealBill - Kate ...
8,7,22,7_risk_reconciliation_the_to,"[risk, reconciliation, the, to, and, accuracy,...",[We are creating a steering committee and proj...
9,8,21,8_deal_price_ect_the,"[deal, price, ect, the, day, this, on, is, fro...","[Subject: Local Production for JuneJohn, I too..."


In [46]:
from umap import UMAP
from hdbscan import HDBSCAN

umap_model = UMAP(n_neighbors=10, n_components=3, min_dist=0.05)
hdbscan_model = HDBSCAN(min_cluster_size=20, min_samples=10,
                        prediction_data=True, gen_min_span_tree=True)

***NOTE**: We need to set `prediction_data=True` otherwise we return **AttributeError: No prediction data was generated***

Then we pass these to the `umap_model` and `hdbscan_model` arguments when initializing `BERTopic`. In the previous notebook we used the `sentence-transformers/all-MiniLM-L6-v2` embedding model, we do the same here by passing this string to the `embedding_model` argument. Finally, we add in the `CountVectorizer` which removes English stopwords for us in the c-TF-IDF step.

In [22]:
#import nltk
#nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/kariato/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [47]:

# we add this to remove stopwords that can pollute topcs
vectorizer_model = CountVectorizer(ngram_range=(1, 2), stop_words=stopwords)

model = BERTopic(
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    embedding_model=embedding_model,
    vectorizer_model=vectorizer_model,
    top_n_words=15,
    language='english',
    calculate_probabilities=True,
    verbose=True
)
#model.fit(data['selftext'])
topics2, probs = model.fit_transform(topics)

2024-03-14 23:58:53,179 - BERTopic - Embedding - Transforming documents to embeddings.
Batches: 100%|██████████| 105/105 [00:22<00:00,  4.70it/s]
2024-03-14 23:59:15,634 - BERTopic - Embedding - Completed ✓
2024-03-14 23:59:15,634 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-03-14 23:59:24,998 - BERTopic - Dimensionality - Completed ✓
2024-03-14 23:59:24,999 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-03-14 23:59:25,143 - BERTopic - Cluster - Completed ✓
2024-03-14 23:59:25,146 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-03-14 23:59:25,984 - BERTopic - Representation - Completed ✓


In [48]:
model.visualize_barchart()

In [49]:
model.visualize_hierarchy()

In [50]:
model.visualize_heatmap()

In [51]:
freq = model.get_topic_info()
print("Number of topics: {}".format( len(freq)))
freq

Number of topics: 4


,Topic,Count,Name,Representation,Representative_Docs
0,-1,34,-1_cows_simulated_risk_results,"[cows, simulated, risk, results, tm, real time...",[Click Here for details . . ._________________...
1,0,3244,0_subject_enron_ect_2001,"[subject, enron, ect, 2001, accounting, origin...","[Sold 10,000 dth of Conger gas to TXU for 4/26..."
2,1,32,1_image_image image_company_equityalert,"[image, image image, company, equityalert, dis...",[es and reports. Please note that links to the...
3,2,27,2_dow_dow jones_jones_points,"[dow, dow jones, jones, points, copyright, jon...",[=20-----Original Message-----Subject: FYI -- ...
